In [1]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
import argparse
import scipy
import torch
import time
import torch.nn as nn
import os
import pandas as pd
import numpy as np
import pickle
import yaml
import matplotlib.pyplot as plt
import torch.nn.functional as F

from fno_utils import FNO2d, FNODatasetSingle

In [3]:
cfg_fn = os.path.join("experiments", f"config_Darcy.yaml")
with open(cfg_fn, "r") as f:
    cfg = yaml.safe_load(f)

In [4]:
pde_name = cfg["filename"].split(".h")[0]
times = []
num_trials = 10
trial_size = 1
train_data = FNODatasetSingle(filename=os.path.join("experiments", cfg["filename"]))
train_loader = torch.utils.data.DataLoader(train_data, batch_size=trial_size)
model_weights = torch.load(os.path.join("experiments", f"{pde_name}_FNO.pt"))

.HDF5 file extension is assumed hereafter


In [5]:
fno = FNO2d(
    num_channels=cfg["num_channels"], 
    modes1=cfg["modes"], 
    modes2=cfg["modes"], 
    width=cfg["width"], 
    initial_step=cfg["initial_step"]).to("cuda")
fno.load_state_dict(model_weights["model_state_dict"])

<All keys matched successfully>